In [ ]:
# We need to create disk space for computing a storing the product- component matrix which is of the order of 60000 x 25700 
#elements. This creates space in the disk for the compute as the RAM cannot handle this size of data computation.This is done
#once and the fie object ( called diskspace) can be amnipulated as a dictionary to add and access data as required. THIS MODULE
#WILL MAKE SPACE IN YOUR HD FOR THE COMPUTATIONS AND THIS CAN BE USED TO STORE ARRAYS, DCTIONARY AND OTHER PYTHON OBJECTS.
import h5py
import numpy as np

PCM = np.zeros((25700, 60000)) # provide the size of array you need for the product- component matrix in the below case
PCMbkup = np.zeros((25700,60000)) # we know it is 25700 x 60000 size. we have to initialize it here 
f= h5py.File('store.hdf5', 'w') # creates a file object of h5py where the array can be stored and manipulated from
f.create_dataset('PCM', data=PCM, chunks = True) # adds a key 'array1' to store the prod- comp- matrix as its value
f.create_dataset('PCMbkup', data=PCMbkup, chunks = True)
f.close()

In [ ]:
import numpy as np
import seaborn as sns
import h5py
component_cost ={}
product_price ={}
np.random.seed(12) # set seed to control random number variation.
#Simulating and adding component costs for 60000 components numbered 0-60000 as per the distributions imputed from excel data.
#In reality this is deterministic data which is already available. Simulating it just for the purposes of our proof of concept
inflexion_C =[5000,13000,28000,46000,55000,59000,60000]
inflexion= [1000, 5000, 15000,23000,25000,25500,25600,25700]
#The inflection points ar computed from real data screenshot for both components as well as the products
#*****************************************************************************************************************************
for i in range(inflexion_C[0]):
    component_cost[i]= [1, 0.002]
for i in range(inflexion_C[0], inflexion_C[1]):
    component_cost[i] = [2, np.random.uniform(.01,.1)]
for i in range(inflexion_C[1], inflexion_C[2]):
     component_cost[i] = [3, np.random.uniform(.1,1)]
for i in range(inflexion_C[2], inflexion_C[3]):
     component_cost[i] = [4, np.random.uniform(1,10)]
for i in range( inflexion_C[3], inflexion_C[4]):
     component_cost[i] = [5, np.random.uniform(10,100)]
for i in range(inflexion_C[4],inflexion_C[5]):
     component_cost[i] = [6, np.random.uniform(100,1000)]
for i in range(inflexion_C[5],inflexion_C[6]):
     component_cost[i] = [7, np.random.uniform(1000,10000)]
with open('component.txt', 'w') as f:
    print(component_cost, file=f)
print("COMPONENT COSTS WRITTEN")        
# Simulating product prices based on product IDs (0- 25700) and their distributions********************************************
for i in range(inflexion[0]):
    product_price[i]= [0, 0.01]
for i in range(inflexion[0], inflexion[1]):
    product_price[i] = [0, np.random.uniform(.01,.1)]
for i in range(inflexion[1], inflexion[2]):
     product_price[i] = [0, np.random.uniform(.1,1)]
for i in range(inflexion[2], inflexion[3]):
     product_price[i] = [0, np.random.uniform(1,10)]
for i in range( inflexion[3], inflexion[4]):
     product_price[i] = [0, np.random.uniform(10,100)]
for i in range(inflexion[4],inflexion[5]):
     product_price[i] = [0, np.random.uniform(100,1000)]
for i in range(inflexion[5],inflexion[6]):
     product_price[i] = [0, np.random.uniform(1000,10000)]
for i in range(inflexion[6],inflexion[7]):
     product_price[i] = [0, np.random.uniform(10000,100000)]
print("PRODUCT PRICES SIMULATED")
# simulate Product component matrix. We know that there are 25700 different products*******************************************
# AND EACH PRODUCT HAS A MAX OF 60,000 COMPONENTS IT COULD BE MANUFACTURED FROM. THIS MATRIX HOLDS THE MAPPING OF THE PRODUCTS 
#TO ITS COMPONENTS. eg P1 CAN BE MADE OF  1 EACH OF COMPONENT 2, 500 AND 58788 THE CPmATRIX HOLDS THIS INFORMATION 
with h5py.File('store.hdf5', 'r+') as f:
    CPMatrix = f['PCM']
    for i in range(len(inflexion)):
        if i ==0:
            lower = 0
        else:
            lower = inflexion[i-1]
        upper = inflexion[i]
        for j in range(lower, upper): # loop through each product
                test ={}
                if i ==0:
                    w= np.random.choice((np.arange(0,inflexion_C[i])), np.random.randint(1,5))
                    w.sort()
                    for k in w:
                        if  k in test.keys():
                            test[k]+=1
                        else:
                            test[k] =1
                    
                    
                                      
                else:
                    for n in range(i):
                        w=[]
                        if n==0:
                            low =0
                        else:
                            low = inflexion_C[n-1]
                        up = inflexion_C[n]
                        w= np.random.choice((np.arange(low,up)), np.random.randint(1,5)*np.minimum(10000,np.power(10,i-n-1)))
                        w.sort()
                        for k in w:
                            if  k in test.keys():
                                test[k]+=1
                            else:
                                test[k] =1
                            
                for keys,val  in test.items():
                    CPMatrix[j,keys]= val
                print(i,j)
f.close()
print("COMPONENT PRODUCT MATRIX WRITTEN") 
def prod(a, b):
    return a*b
sumprod = np.vectorize(prod)


with h5py.File('store.hdf5', 'r+') as f:
    d1 = f['PCM']
    
    for i in range(inflexion[-1]):
        print(i, len(d1[i]),"cost update", sum(d1[i]), max(d1[i]), min(d1[i]), sum(d1[i])/len(d1[i]) )
        product_price[i][0]= sumprod(d1[i],[v[1] for k,v in component_cost.items()] )
        
f.close()    
    
   
    
print("PRODUCT COST COMPUTED")  


#Making sure the product price is greater than its costs
#for k,v in product_price.items():
    
    #while(product_price[k][1]<=product_price[k][0]):
        #product_price[k][1] = product_price[k][0] + 0.2*np.random.normal(product_price[k][0], 0.5*product_price[k][0])
        
with open('product-price.txt', 'w') as f:
    print(product_price, file=f)  
print("PRODUCT COST AND PRICES UPDATED AND WRITTEN")
#sns.set_style('darkgrid')
#sns.distplot(margin)
print("PRODUCT COST AND PRICES UPDATED AND WRITTEN")

In [1]:
import h5py
import numpy as np
from scipy import stats
import scipy.linalg as la
SIM =2000
Scenarios = np.zeros((SIM, 25700)) # provide the size of array you need for the scenarios matrix in the below case
# we know it is 2000 x 25700 size. we have to initialize it here 
f= h5py.File('store-scen.hdf5', 'w') # creates a file object of h5py where the array can be stored and manipulated from
f.create_dataset('Scenarios', data=Scenarios, chunks = True) 
f.close()

In [3]:
import h5py
import numpy as np
from scipy import stats
import scipy.linalg as la
SIM =2000
# Simulating demand parameters (shape and scale) for each category of products. Assume the first category has a demand U~(1M - 10M)
#and the rest follow a decrease of an order of magnitude as the category increase on an average. The top 100 most expensive products
# we will assume a demand correlation based on differing marginals and the use of a copula as an illustration of using demands
#beig generated by differnt distributions ( not all being normal) anbd them being correlated as is the case in the real wrld
# for the process of simplicity we will consider the folllowing 5 differnt distributions we will sample from:-uniform,exponential
#  normal, gumbel and gamma
sample={}
inflexion= [1000, 5000, 15000,23000,25000,25500,25600,25700]
params={}
np.random.seed(1773) #CTRL RANDOM SEED 
# 1- Uniform, 2 Exponential 3 Normal 4 Gumbel 5 Gamma
def chooseDist(d,K):
    b=[]
    if d==1:
        b.extend([1,K, 9*K])
    elif d==2:
        b.extend([2,K,stats.uniform.rvs(K,9*K)])
    elif d==3:
        b.append(3)
        b.append(stats.uniform.rvs(K,9*K))
        b.append(stats.uniform.rvs(K,9*K))
    elif d ==4:
        b.extend([4,stats.uniform.rvs(K,9*K)])
        b.append(b[1]/10)
    else:
        b.append(5)
        b.append(stats.uniform.rvs(K,9*K))
        b.append(b[1])
    return b
        

            
def createSample(array, N):
    bb=[]
    if array[0]==1:
        bb=(stats.uniform.rvs(array[1], array[2], N))
    elif array[0]==2:
        bb=(stats.expon.rvs(array[1], array[2], N))
    elif array[0]==3:
        bb=(stats.norm.rvs(array[1], array[2], N))
    elif array[0]==4:
        bb=(stats.gumbel_r.rvs(array[1], array[2], N))
    else:
        bb=(stats.gamma.rvs(2,array[1], array[2], N))
    bb = [np.maximum(x,0) for x in bb]
    return bb
    
    
def createcorSample(array,array2):
    bb=[]
    if array[0]==1:
        bb=(stats.uniform.ppf(array2, array[1], array[2]))
    elif array[0]==2:
        bb=(stats.expon.ppf(array2, array[1], array[2]))
    elif array[0]==3:
        bb=(stats.norm.ppf(array2,array[1], array[2]))
    elif array[0]==4:
        bb=(stats.gumbel_r.ppf(array2, array[1], array[2]))
    else:
        bb=(stats.gamma.ppf(array2, 2,array[1], array[2]))
    bb = [np.maximum(x,0) for x in bb]
    return bb
    
# create a random correlation structure for copula to take care of the 100 most expenbsive and correlated products.
#you can edit this and prvide your own correlation structure.
N =100
high =200 # random number you can use anything you wish as the max random int to be generated
mean=np.zeros(N) # or you can provide mean = a N array with the expected values of the random variables.
v =np.ones(N) # or you can provide std =a N array with the std dev of of the random variables.( 1 is for std normal generation)
#***********************************************************************************************************************
eigsim = np.random.randint(high, size=(N,))
eigsim = N*eigsim/sum(eigsim) # this tries t0 ensure its a psd matrix but sometimes floating point failures happen.
                             #change seed and re run
x = stats.random_correlation.rvs((eigsim)) #random correlation generator
matmul= (np.dot(v[:,None],v[None,:]))
df_cov = np.multiply(matmul,x)
#print(df_cov)
mvnorm = stats.multivariate_normal(mean,df_cov )
# Generate random samples from multivariate normal with correlation randomly generated as shown by matrix 'x'
x = mvnorm.rvs(SIM)
x=x.T
print(len(x))
print("RANDOM CORRELATION STRUCTURE CREATED") 


uniftrans= np.empty((100,SIM)) # since we are doing 100 products that are correlated demands using a copula
c=0
for h in x[:]:
    uniftrans[c] = stats.norm.cdf(h)
    c+=1

with h5py.File('store-scen.hdf5', 'r+') as f:
    d10 = f['Scenarios']
    for i in range(len(inflexion)):
        if i ==0:
            lower = 0
        else:
            lower = inflexion[i-1]
        upper = inflexion[i]
        K = np.maximum( 100,np.minimum(1000000, np.power(10, len(inflexion)-i)) )
        for j in range(lower, upper):
            d= np.random.choice([1,2,3,4,5])
            params[j]= chooseDist(d,K)
            if ( j < 25600): #HARDCODED NUMBERS IN THIS PART OF CODE 
                sample[j] = createSample(params[j],SIM)
            else:               
                sample[j]= createcorSample(params[j], uniftrans[j-25600])
    print("SAMPLES CALCULATED")
    for x in range(inflexion[-1]):
        if x%100==0:
            print(x/100)
        for w in range(len(sample[x])):
             d10[w,x]=sample[x][w]
    print("SCENARIOS WRITTEN")
f.close()                    



100
RANDOM CORRELATION STRUCTURE CREATED
SAMPLES CALCULATED
0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
